In [1]:
%pip install pandas
%pip install numpy
%pip install scikit-learn

  Using cached pandas-2.2.3-cp310-cp310-win_amd64.whl (11.6 MB)
     ---------------------------------------- 12.9/12.9 MB 9.3 MB/s eta 0:00:00
  Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
     ------------------------------------- 346.8/346.8 kB 10.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



     --------------------------------------- 11.1/11.1 MB 11.3 MB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
     ---------------------------------------- 43.9/43.9 MB 7.0 MB/s eta 0:00:00
  Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)



[notice] A new release of pip available: 22.2.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import joblib  

In [3]:
df = pd.read_csv("Datasets/plant_growth_data.csv")
df.head()

,Soil_Type,Sunlight_Hours,Water_Frequency,Fertilizer_Type,Temperature,Humidity,Growth_Milestone
0,loam,5.192294,bi-weekly,chemical,31.719602,61.591861,0
1,sandy,4.033133,weekly,organic,28.919484,52.422276,1
2,loam,8.892769,bi-weekly,none,23.179059,44.660539,0
3,loam,8.241144,bi-weekly,none,18.465886,46.433227,0
4,sandy,8.374043,bi-weekly,organic,18.128741,63.625923,0


In [4]:
# Encoding Fitur Kategorikal
label_encoders = {}
for column in ["Soil_Type", "Water_Frequency", "Fertilizer_Type"]:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le


In [5]:
# Fitur dan Target
X = df.drop("Growth_Milestone", axis=1)
y = df["Growth_Milestone"]

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Model Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [7]:
# Evaluasi Model
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
f"RMSE: {rmse:.2f}"

'RMSE: 0.55'

In [8]:
joblib.dump(model, "Model/random_forest_plant_growth.pkl")

['Model/random_forest_plant_growth.pkl']

In [9]:
loaded_model = joblib.load("Model/random_forest_plant_growth.pkl")

# Fungsi Prediksi Kustom
def predict_custom(input_data):
    # Format data menjadi DataFrame
    input_df = pd.DataFrame([input_data])

    # Encode fitur kategorikal
    for column, le in label_encoders.items():
        if column in input_df:
            input_df[column] = le.transform(input_df[column])

    # Imputasi nilai yang hilang
    imputer = SimpleImputer(strategy="mean")
    input_df = pd.DataFrame(imputer.fit_transform(input_df), columns=X.columns)

    # Prediksi
    prediction = loaded_model.predict(input_df)[0]
    return f"Probabilitas bertahan hidup: {prediction * 100:.2f}%"

In [14]:
predict_custom({
    "Soil_Type": "sandy",
    "Sunlight_Hours": 5.0,
    "Water_Frequency": "weekly",
    "Fertilizer_Type": "organic",
    "Temperature": 38.0,
    "Humidity": 50.0, 
})

'Probabilitas bertahan hidup: 68.00%'